## Model evaluation

In [1]:
from typing import Mapping
from typing import Optional
from functools import partial
import os
import torch
import numpy as np
import tqdm.notebook as tqdm
from sdc.metrics import SDCLoss
from sdc.config import build_parser
from sdc.oatomobile.torch.baselines import init_rip
from sdc.oatomobile.torch.baselines.robust_imitative_planning import load_rip_checkpoints
from ysdc_dataset_api.dataset import MotionPredictionDataset
from ysdc_dataset_api.evaluation.utils import load_submission_proto, save_submission_proto
from ysdc_dataset_api.evaluation import Submission, object_prediction_from_model_output, save_submission_proto
from sdc.oatomobile.torch.baselines import batch_transform


/home/master/10/cytseng/miniconda3/envs/sdc/lib/python3.8/site-packages/ysdc_dataset_api/utils/transform.py:90: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, C))
  res = transform @ ph
/home/master/10/cytseng/miniconda3/envs/sdc/lib/python3.8/site-packages/numba/core/typing/npydecl.py:965: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, C))
  warnings.warn(NumbaPerformanceWarning(msg))


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
evaluation_dataset_path = \
'/home/master/10/cytseng/shifts_sdc_dataset/canonical-eval-data/evaluation'
prerendered_dataset_path = \
'/home/master/10/cytseng/shifts_sdc_dataset/canonical-eval-data/evaluation_renders'

# Submission_dir
submission_dir = '/home/master/10/cytseng/shifts_eval/shifts/sdc/submissions_eval/'

In [4]:
evaluation_dataset = MotionPredictionDataset(
    dataset_path=evaluation_dataset_path,
    prerendered_dataset_path=prerendered_dataset_path,
)

In [5]:
evaluation_dataset.num_scenes

36804

In [7]:
parser = build_parser()
args = parser.parse_args('')


def ipynb_patch_args(args):
    args.dir_checkpoint = '/home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints'

    # The below configuration was our best performing in baseline experiments.
    # See paper for more details and the configurations considered.
    
    # Backbone model details
    # Behavioral Cloning: 
    # MobileNetv2 feature encoder, GRU decoder
    args.model_name = 'bc'
    args.model_dim_hidden = 512
    args.exp_device = 'cuda:0'
    
    # Used in scoring generated trajectories and obtaining 
    # per-plan/per-scene confidence scores.
    # See 
    #   `sdc.oatomobile.torch.baselines.robust_imitative_planning.py` 
    # for details.
    args.rip_per_plan_algorithm = 'MA'
    args.rip_per_scene_algorithm = 'MA'
    
    # Number of ensemble members
    args.rip_k = 5
    
    # Data loading
    # https://pytorch.org/docs/stable/data.html
    args.exp_batch_size = 512
    args.data_num_workers = 4
    args.data_prefetch_factor = 2
    
    # Cache loss metrics here
    args.dir_metrics = '/home/master/10/cytseng/shifts_sdc_dataset/data/metrics'

    return args

c = ipynb_patch_args(args)

In [8]:
# Softmax for normalize
def _softmax_normalize(weights: np.ndarray) -> np.ndarray:
    weights = np.exp(weights - np.max(weights))
    return weights / weights.sum(axis=0)

In [10]:
class Model:
    def __init__(self, c):
        self.c = c
        # Initialize torch hub dir to cache MobileNetV2
        torch.hub.set_dir(f'{c.dir_checkpoint}/torch_hub')
        
    def load(self):
        model, self.full_model_name, _, _ = init_rip(c=self.c)
        checkpoint_dir = f'{c.dir_checkpoint}/{self.full_model_name}'
        self.model = load_rip_checkpoints(
            model=model, device=c.exp_device, k=c.rip_k,
            checkpoint_dir=checkpoint_dir)
        
    
    def predict(self, batch: Mapping[str, torch.Tensor], sdc_loss: Optional[SDCLoss]):
        """
        Args:
            batch: Mapping[str, torch.Tensor], with 'feature_maps' key/value

        Returns:
            Sequence of dicts. Each has the following structure:
                {
                    predictions_list: Sequence[np.ndarray],
                    plan_confidence_scores_list: Sequence[np.ndarray],
                    pred_request_uncertainty_measure: float,
                }
        """
        self.model.eval()
        with torch.no_grad():
            predictions, plan_confidence_scores, pred_request_confidence_scores = (
                self.model(**batch))
            
        predictions = predictions.detach().cpu().numpy()
#         print("predictions: ", predictions)

        plan_confidence_scores = plan_confidence_scores.detach().cpu().numpy()
#         for i in range(predictions.shape[0]):
#             plan_confidence_scores[i] = _softmax_normalize(plan_confidence_scores[i])
#         print("plan_confidence_scores: ", plan_confidence_scores)

        pred_request_confidence_scores = pred_request_confidence_scores.detach().cpu().numpy()
#         print("pred_request_confidence_scores: ", pred_request_confidence_scores)

        if sdc_loss is not None:
            ground_truth = batch['ground_truth_trajectory'].detach().cpu().numpy()
            sdc_loss.cache_batch_losses(
                predictions_list=predictions,
                ground_truth_batch=ground_truth,
                plan_confidence_scores_list=plan_confidence_scores,
                pred_request_confidence_scores=pred_request_confidence_scores)
        
        return [
            {
                'predictions_list': predictions[i],
                'plan_confidence_scores_list': plan_confidence_scores[i],
                # Negate, as we need to provide an uncertainty measure
                # for the submission pb, not a confidence score.
                'pred_request_uncertainty_measure':
                    -(pred_request_confidence_scores[i])
            } for i in range(predictions.shape[0])]

# Initialize and load ensemble of k models from checkpoints
# On first run, will fail and create a directory where checkpoints
# should be placed.
model = Model(c=c)
model.load()

RIP kwargs:
{'cache_all_preds': False,
 'device': 'cuda:0',
 'k': 5,
 'model_name': 'bc',
 'num_preds': 5,
 'per_plan_algorithm': 'MA',
 'per_scene_algorithm': 'MA',
 'samples_per_model': 10}
Building RIP agent with backbone model bc, per-plan algorithm MA, per-scene algorithm MA, 5 ensemble members.
Model kwargs:
{'bc_deterministic': False,
 'device': 'cuda:0',
 'dim_hidden': 512,
 'generation_mode': 'sampling',
 'in_channels': 17,
 'output_shape': (25, 2)}
BC Model: using Gaussian likelihood.
BC Model: using generation mode sampling.


Using cache found in /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/torch_hub/pytorch_vision_v0.9.0


Model kwargs:
{'bc_deterministic': False,
 'device': 'cuda:0',
 'dim_hidden': 512,
 'generation_mode': 'sampling',
 'in_channels': 17,
 'output_shape': (25, 2)}
BC Model: using Gaussian likelihood.
BC Model: using generation mode sampling.
Model kwargs:
{'bc_deterministic': False,
 'device': 'cuda:0',
 'dim_hidden': 512,
 'generation_mode': 'sampling',
 'in_channels': 17,
 'output_shape': (25, 2)}


Using cache found in /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/torch_hub/pytorch_vision_v0.9.0
Using cache found in /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/torch_hub/pytorch_vision_v0.9.0


BC Model: using Gaussian likelihood.
BC Model: using generation mode sampling.
Model kwargs:
{'bc_deterministic': False,
 'device': 'cuda:0',
 'dim_hidden': 512,
 'generation_mode': 'sampling',
 'in_channels': 17,
 'output_shape': (25, 2)}
BC Model: using Gaussian likelihood.
BC Model: using generation mode sampling.
Model kwargs:
{'bc_deterministic': False,
 'device': 'cuda:0',
 'dim_hidden': 512,
 'generation_mode': 'sampling',
 'in_channels': 17,
 'output_shape': (25, 2)}


Using cache found in /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/torch_hub/pytorch_vision_v0.9.0
Using cache found in /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/torch_hub/pytorch_vision_v0.9.0


BC Model: using Gaussian likelihood.
BC Model: using generation mode sampling.
Loaded ensemble member 1 from path /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/rip-bc-k_5-plan_ma-scene_ma/model-seed-90-epoch-83.pt
Loaded ensemble member 2 from path /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/rip-bc-k_5-plan_ma-scene_ma/model-seed-10-epoch-57.pt
Loaded ensemble member 3 from path /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/rip-bc-k_5-plan_ma-scene_ma/model-seed-20-epoch-90.pt
Loaded ensemble member 4 from path /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/rip-bc-k_5-plan_ma-scene_ma/model-seed-60-epoch-64.pt
Loaded ensemble member 5 from path /home/master/10/cytseng/shifts_eval/shifts/sdc/model_checkpoints/rip-bc-k_5-plan_ma-scene_ma/model-seed-40-epoch-78.pt
Successfully loaded all 5 ensemble members.


In [11]:
# Init dataloader
dataloader_kwargs = {
    'batch_size': c.exp_batch_size,
    'num_workers': c.data_num_workers,
    'prefetch_factor': c.data_prefetch_factor,
    'pin_memory': True
}

print(f'Building dataloaders with kwargs {dataloader_kwargs}.')
evaluation_dataloader = torch.utils.data.DataLoader(evaluation_dataset, **dataloader_kwargs)

Building dataloaders with kwargs {'batch_size': 512, 'num_workers': 4, 'prefetch_factor': 2, 'pin_memory': True}.


### Produce a Submission Protobuf

In [12]:
submission = Submission()

batch_cast = partial(
    batch_transform, device=c.exp_device, downsample_hw=None,
    data_use_prerendered=True)

for batch_id, batch in enumerate(tqdm.tqdm(evaluation_dataloader)):
    batch = batch_cast(batch)
    batch_output = model.predict(batch, None)

    for i, data_item_output in enumerate(batch_output):
        proto = object_prediction_from_model_output(
            track_id=batch['track_id'][i],
            scene_id=batch['scene_id'][i],
            model_output=data_item_output,
            is_ood=False)  # Set fake value as we do not know is_ood for evaluation data.

        submission.predictions.append(proto)

0it [00:00, ?it/s]

In [13]:
# First, let's write out a submission protobuf (as one should submit for the competition).
save_submission_proto('eval_moscow_and_ood_submission.pb', submission=submission)
# Can check that things were written correctly as follows:
new_sub = load_submission_proto('eval_moscow_and_ood_submission.pb')
len(new_sub.predictions)

512652

In [14]:
# First, let's write out a submission protobuf (as one should submit for the competition).
save_submission_proto(submission_dir \
                      + '/'+ model.full_model_name + '_submission.pb', submission=submission)

In [15]:
# Can check that things were written correctly as follows:
new_sub = load_submission_proto(submission_dir +\
                                  '/'+ model.full_model_name + '_submission.pb')
len(new_sub.predictions)

512652